In [1]:
import Agents
import Markets

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats

def add_price_data(df, t):
    
    #pA_buy, pA_sell, pB_buy, pB_sell = df
    p_buy, p_sell = df
    
    for v, agent in enumerate(agent_list):
        p_buy[t][v] = agent.p_buy
        p_sell[t][v] = agent.p_sell



In [2]:
def add_quant_data(df, t):

    for v,agent in enumerate(agent_list):

        df[t][v] = agent.Q
    
    
    #print(agent_list[0].Q)
    
def add_deficit_data(df, t):

    for v,agent in enumerate(agent_list):

        df[t][v] = agent.D
    
def add_money_data(df, t):
    
    for v,agent in enumerate(agent_list):

        df[t][v] = agent.M
    
def reset():
    
    for agent in agentA_list:
        
        #if initialize_money=='random':
        #    agent.M = np.random.uniform(M)
        #else:
        #    agent.M = M
        #    
        #if initialize_quantity=='random':
        #    agent.Q[0] = np.random.uniform(QA1)
        #else:
        #    agent.Q[0] = QA1

        if initialize_prices=='random':
            
            agent.p_buy  = np.random.uniform(agent.M, size=n)
            agent.p_sell = np.random.uniform(agent.M, size=n)
            
        else:
            agent.p_buy  = [p1buy_A_0, p2buy_A_0]
            agent.p_sell = [p1sell_A_0, p2sell_A_0]
        
    for agent in agentB_list:
        
        #if initialize_money=='random':
        #    agent.M = np.random.uniform(M)
        #else:
        #    agent.M = M
        
        if initialize_prices=='random':
            agent.p_buy  = np.random.uniform(agent.M, size=n)
            agent.p_sell = np.random.uniform(agent.M, size=n)
        
        else:
            agent.p_buy  = [p1buy_B_0, p2buy_B_0]
            agent.p_sell = [p1sell_B_0, p2sell_B_0]
    
    market.reset()


In [3]:
def plot_market(i, *, ps=10, log=False):

    f, ax = plt.subplots(1, figsize=(10,4)) #(2,1)

    ax.plot(ME_df[:,i], c='C0', label=f'$\Delta M_{i}$')
    ax.plot(QE_df[:,i], c='C1', label=f'$\Delta Q_{i}$')
    ax.plot(ntries_df, c='k', label='Tries', lw='0.5')

    p = ME_df[:,i]/QE_df[:,i]
    ax.scatter(np.arange(0,T), p, c='g', label=f'$p_{i}$', s=ps)

    ax.legend()
    
    if log:
        plt.yscale('log')
        
    plt.title(f'Market for Commodity {i}')
    plt.xlabel('t')
    plt.ylabel('$M$, $Q$')


def plot_prices(p_df, i, *, A=0, B=1, ps=10, log=False, includeM=True):

    'A sells i=0 and buys i=1, and vice versa for B'

    if i == 0:
        plt.plot(p_df[1][:,A,i], c='C0', ls='-.', label=f"A's Selling Price for {i}")
        plt.plot(p_df[0][:,B,i], c='C1', ls='-', label=f"B's Buying Price for {i}")

    elif i == 1:
        plt.plot(p_df[0][:,A,i], c='C0', ls='-', label=f"A's Buying Price for {i}")
        plt.plot(p_df[1][:,B,i], c='C1', ls='-.', label=f"B's Selling Price for {i}")

    if includeM:
        plt.plot(M_df[:,A], c='C0', ls=':', lw=0.5, label=f'$M_A$')
        plt.plot(M_df[:,B], c='C1', ls=':', lw=0.5, label=f'$M_B$')
    
    p = ME_df[:,i]/QE_df[:,i]
    plt.scatter(np.arange(T)+0.5, p, c='g', s=ps)
    
    plt.legend()
    
    if log:
        plt.yscale('log')

    plt.title(f'Market Prices for Commodity {i}')
    plt.xlabel('t')
    plt.ylabel('Prices')
    
def plot_quantities(Q_df, D_df, i, *, A=0, B=1):

    plt.plot(Q_df[:,A,i], c='C0', ls='-', label=f"A's Q{i}")
    plt.plot(D_df[:,A,i], c='C0', ls=':', label=f"A's D{i}")

    plt.plot(Q_df[:,B,i], c='C1', ls='-', label=f"B's Q{i}")
    plt.plot(D_df[:,B,i], c='C1', ls=':', label=f"B's D{i}")

    plt.legend()

    plt.title(f'Quantities for Commodity {i}')
    plt.xlabel('t')
    plt.ylabel('Q,D')


In [4]:

def run_market():

    reset()

    # Setup data
    ME_df = np.zeros((T,n))
    QE_df = np.zeros((T,n))
    sE_df = np.zeros((T,n))
    ntries_df = []

    # Run market
    for t in range(T-1):

        # Market action occurs at half-time interval
        ME, QE, stn_dev, ntries = market.run_exchange(ExchangeDict)

        # Reset quantities
        for agentA in agentA_list:
            agentA.Q = QA.copy()
            agentA.M = M

        for agentB in agentB_list:
            agentB.Q = QB.copy()
            agentB.M = M


        # Market action occurs at half-time interval
        ME_df[t] = ME
        QE_df[t] = QE
        sE_df[t] = stn_dev
        ntries_df.append(ntries)

    p = np.nanmean(ME_df/QE_df, axis=0)

    return p

In [5]:
savefig=True

In [6]:
T = 100
B = 10 # bootstrap number of market simulations per iteration

price_adjust_method = 'step_random'

initialize_money = 'set'
M = 20

NA = 1

n = 2

QA1 = 10
QA2 = 0

QB1 = 0
QB2 = 0

initialize_prices = 'random'


QA_0 = np.array([QA1, QA2])
QB_0 = np.array([QB1, QB2])

QA = np.copy(QA_0)
QB = np.copy(QB_0)

qA = np.array([0, 0])
qB = np.array([0, 0])

cA = np.array([0, 0])
cB = np.array([1, 0])

DA = cA.copy()
DB = cB.copy()

dN = 1
NB_list = np.arange(1, 50+dN, dN)

NB_list = np.array(NB_list)

In [7]:
verbose = False
alpha = 0.5
beta = 2.0
gamma = 1.00
epsilon_switch = 0.00
epsilon_rand = 0.00
max_agent_tries = 1
min_price = 1e-5

for max_agent_tries in [1,2,10]:
    for alpha in [0.0, 0.1, 1.0]:
        for beta in [0.0, 1.0, 0.1, 1.0]:
            for epsilon_rand in [0.00, 0.01]:

                p_list = []
                ps_list = []

                for NB in NB_list:

                    NB = 1

                    N = NA+NB

                    agentA_list = [Agents.Agent(QA.copy(), DA.copy(), M, n_actions=1, input_dims=[1]) for agent in range(NA)]
                    agentB_list = [Agents.Agent(QB.copy(), DB.copy(), M, n_actions=1, input_dims=[1]) for agent in range(NB)]

                    agent_list = agentA_list + agentB_list

                    max_tries = N*max_agent_tries
                    market = Markets.Market(n, 
                                            verbose=verbose, 
                                            minimum_price=min_price, 
                                            alpha=alpha, 
                                            beta=beta, 
                                            gamma=gamma, 
                                            epsilon_switch=epsilon_switch, 
                                            epsilon_rand=epsilon_rand,
                                            max_agent_tries=max_agent_tries, 
                                            max_tries=max_tries,
                                            price_adjust_method=price_adjust_method)


                    ExchangeDict = {agent : 2 for agent in agent_list}

                    p_boot = []
                    for boot_num in range(B):

                        ## RUN MARKET SIMULATION ##
                        p = run_market()
                        ###########################

                        p_boot.append(p[0])

                    p0 = np.nanmean(p_boot)
                    ps = np.nanstd(p_boot)

                    p_list.append(p0)
                    ps_list.append(ps)

                p_list = np.array(p_list)
                ps_list = np.array(ps_list)

                f,ax = plt.subplots()

                ax.plot(NB_list, p_list, c='C0')

                ax.plot(NB_list, p_list+ps_list, ls=':', c='C0', lw=0.5)
                ax.plot(NB_list, p_list-ps_list, ls=':', c='C0', lw=0.5)

                ax.set_xlim(0, NB_list.max())
                ax.set_ylim(0,M)

                ax.set_xlabel('Number of Buyers')
                ax.set_ylabel('Price')

                ax.set_title(rf'Number of Sellers={NA}  $\alpha=${alpha}  $\beta=${beta}  $\epsilon_r=${epsilon_rand} num_tries={max_agent_tries}')

                if savefig:
                    savetitle = rf'Adjust Buyers - Number of Sellers={NA}  alpha={alpha:.2f}  beta={beta:.2f}  epsilon_r={epsilon_rand:.2f} num_tries={max_agent_tries}.png'
                    plt.savefig(f'Adjust Buyers/Experiment 02/Num Agent Tries {max_agent_tries}/' + savetitle)
                    plt.close(f)

C:\Users\Alex\AppData\Local\Temp\ipykernel_24220\1769230898.py:33: RuntimeWarning: invalid value encountered in divide
  p = np.nanmean(ME_df/QE_df, axis=0)
C:\Users\Alex\AppData\Local\Temp\ipykernel_24220\1769230898.py:33: RuntimeWarning: Mean of empty slice
  p = np.nanmean(ME_df/QE_df, axis=0)


In [8]:
#p_list, ps_list

In [9]:
ps_list


array([2.20119002, 1.72794019, 2.70902342, 1.79843725, 2.86859043,
       2.67833025, 3.81774358, 3.07585162, 0.9421298 , 2.60532703,
       2.71984669, 3.37128832, 1.95876961, 2.63820394, 3.02127491,
       2.33813089, 2.42257299, 2.27564529, 3.30381848, 2.93015759,
       1.66090907, 2.06446092, 1.87439685, 3.15313354, 2.94336384,
       1.76759314, 2.95812759, 3.05305796, 2.25219132, 2.20892935,
       1.6537515 , 2.71807362, 3.05440329, 3.08781975, 2.57862812,
       1.71514835, 3.91624468, 3.33469988, 2.84646179, 2.36193388,
       1.992933  , 2.79763359, 1.88001918, 2.29016135, 3.55055398,
       2.31337113, 1.7512256 , 1.52887429, 2.62784798, 2.08932462])